In [1]:
from datasets import load_dataset

dataset = load_dataset("yelp_review_full")

In [2]:
dataset

DatasetDict({
    train: Dataset({
        features: ['label', 'text'],
        num_rows: 650000
    })
    test: Dataset({
        features: ['label', 'text'],
        num_rows: 50000
    })
})

In [3]:
dataset["train"][111]

{'label': 2,
 'text': "As far as Starbucks go, this is a pretty nice one.  The baristas are friendly and while I was here, a lot of regulars must have come in, because they bantered away with almost everyone.  The bathroom was clean and well maintained and the trash wasn't overflowing in the canisters around the store.  The pastries looked fresh, but I didn't partake.  The noise level was also at a nice working level - not too loud, music just barely audible.\\n\\nI do wish there was more seating.  It is nice that this location has a counter at the end of the bar for sole workers, but it doesn't replace more tables.  I'm sure this isn't as much of a problem in the summer when there's the space outside.\\n\\nThere was a treat receipt promo going on, but the barista didn't tell me about it, which I found odd.  Usually when they have promos like that going on, they ask everyone if they want their receipt to come back later in the day to claim whatever the offer is.  Today it was one of th

In [4]:
import random
import pandas as pd
import datasets
from IPython.display import display, HTML

In [5]:
def show_random_elements(dataset, num_examples=10):
    assert num_examples <= len(dataset), "Can't pick more elements than there are in the dataset."
    picks = []
    for _ in range(num_examples):
        pick = random.randint(0, len(dataset)-1)
        while pick in picks:
            pick = random.randint(0, len(dataset)-1)
        picks.append(pick)

    df = pd.DataFrame(dataset[picks])
    for column, typ in dataset.features.items():
        if isinstance(typ, datasets.ClassLabel):
            df[column] = df[column].transform(lambda i: typ.names[i])
    display(HTML(df.to_html()))

In [6]:
show_random_elements(dataset["train"])

,label,text
0,3 stars,"I've come to this location several times and never had any problems. They are always fast and have my order right. Even during their happy hour.\n\nJust took a bite of one of my tacos and got a mouth full of clear, ice cold lettuce. It's one thing for cold lettuce, but frozen lettuce? Really Taco Bell?! I had to take a star back for that one."
1,2 star,"Another buffet deal from buffet of buffet, but this one not my favorite, the food for buffet just too so so.\nService was good and nice view of the flamingos, that's it."
2,1 star,"Another disgusting tasting Chinese joint in Phoenix. My order was simple - steamed dumplings and wonton soup. The meat inside the filling tasted like something very questionable - not pork, not chicken, not beef. Something dark and oily and ominous. I could not eat either dish as both tasted as if they were filled with meat from the same mystery animal.\n\nWaste of money. So I ended up eating a small portion of a visiting relative's chicken and shrimp combo which was almost devoid of meat (might be a good thing, actually) and had absolutely no spice or kick or flavor. \n\nWill not be back."
3,3 stars,Cute and small museum. Good for a 15 minute break. Good for reminiscing. Kids would be bored. The gift shop had a pretty good selection of old fashion toys and books. It is a well done little museum though!
4,1 star,Awful food and service!\n\nI always try a place twice before I write a review so I gave this place two chances and it is an unorganized mess. It took 38 min to get the wrong sandwich TWICE. There were two girls making the sandwiches and handing them to the poor girl at the register who clearly was a newbie. They would hand her the sandwiches and say here....when she would ask what they were they would look annoyed and roll their eyes. The poor girl was just trying to make sure the right order went out the the right people. Also for this being a bakery why was my bread nasty and stale both times??? Super gross. My bread tasted like it was a week old. The first time I had the Turkey avocado sandwich which was BATHED in mayo. The second time I had the club which had mayo all over the outside of the sandwich in globs and the bacon appeared to be semi-raw.....not thanks....do not recommend this place.
5,3 stars,"This chain of faux-Mexican fast food joints will always hold a special place in my heart. After a night on the strip, my buddies and I were looking for something quick and good. This is not what came to mind for everyone else. I couldn't wait to have some chicken soft tacos! They had Raising canes in mind.\n\nIt was in sight and only after we arrived at the drive-thru and they were closed, we proceeded over to Del Taco.\n\nWe ordered $30 worth of food and scarfed it down licketly-split. The \""beef\"" is sub-par but the chicken products are outstanding. Spicy Jack Chicken Quesadilla, the Chicken Soft Tacos, and Chicken Burritos. Brilliant!"
6,5 stars,"Love this place! I have eaten here many times with my family. The service is always great. The owner takes notice of repeat customers and frequently stops by the table to say hello. The Pork Street Tacos, Chicken Quesadillas and Chicken Burritos are all amazing. I have tried all of the salsas in their \""Salsa Bar\"" and they are great as well. \n\nI also really enjoy the casual laid back atmosphere with TV's that play various sports. They have a great outdoor patio area that is nice when the weather is cooler (although I think the furniture out there could be upgraded a bit).\n\nOn the weekends there is a cool vibe and it turns into a busy \""watering hole\"" with great food.\n\nWe will definitely eat here again... and again... :)"
7,2 star,"The food at this place makes me sad and the restaurant and the service itself make me even sadder. It's a new restaurant so I think they need to work out a few kinks, like the overhead lighting and the super quiet interior. It was so quiet, bright and harsh in this place, I felt ov

In [7]:
from transformers import AutoTokenizer
#use checkpoint 
model_name = "models/bert-base-cased-finetune-yelp/checkpoint-60500"


tokenizer = AutoTokenizer.from_pretrained("bert-base-cased")


def tokenize_function(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True)


tokenized_datasets = dataset.map(tokenize_function, batched=True)

In [8]:
show_random_elements(tokenized_datasets["train"], num_examples=1)

,label,text,input_ids,token_type_ids,attention_mask
0,3 stars,"So this review is for the airport alone. It seems that they are always crowded, and they don't make my airline put the connections next to each other, but they are relatively clean. Overall I like the airport, but I'm one who likes to eat at restaurants, and the restaurant selection is limited to me.","[101, 1573, 1142, 3189, 1110, 1111, 1103, 3871, 2041, 119, 1135, 3093, 1115, 1152, 1132, 1579, 11090, 117, 1105, 1152, 1274, 112, 189, 1294, 1139, 8694, 1508, 1103, 6984, 1397, 1106, 1296, 1168, 117, 1133, 1152, 1132, 3860, 4044, 119, 8007, 146, 1176, 1103, 3871, 117, 1133, 146, 112, 182, 1141, 1150, 7407, 1106, 3940, 1120, 7724, 117, 1105, 1103, 4382, 4557, 1110, 2609, 1106, 1143, 119, 102, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...]","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...]","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...]"


### 数据抽样

使用全个数据样本，在 BERT 上演示小规模训练（基于 Pytorch Trainer）

In [9]:
full_train_dataset = tokenized_datasets["train"]
full_eval_dataset = tokenized_datasets["test"]

In [10]:
from transformers import AutoModelForSequenceClassification

model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=5)

In [11]:
from transformers import TrainingArguments

model_dir = "models/bert-base-cased-finetune-yelp"

# logging_steps 默认值为500，根据我们的训练数据和步长，将其设置为100
training_args = TrainingArguments(output_dir=model_dir,
                                  per_device_train_batch_size=32,
                                  num_train_epochs=5,
                                  logging_steps=100)

In [12]:
# 完整的超参数配置
print(training_args)

TrainingArguments(
_n_gpu=1,
accelerator_config={'split_batches': False, 'dispatch_batches': None, 'even_batches': True, 'use_seedable_sampler': True, 'gradient_accumulation_kwargs': None},
adafactor=False,
adam_beta1=0.9,
adam_beta2=0.999,
adam_epsilon=1e-08,
auto_find_batch_size=False,
bf16=False,
bf16_full_eval=False,
data_seed=None,
dataloader_drop_last=False,
dataloader_num_workers=0,
dataloader_persistent_workers=False,
dataloader_pin_memory=True,
dataloader_prefetch_factor=None,
ddp_backend=None,
ddp_broadcast_buffers=None,
ddp_bucket_cap_mb=None,
ddp_find_unused_parameters=None,
ddp_timeout=1800,
debug=[],
deepspeed=None,
disable_tqdm=False,
dispatch_batches=None,
do_eval=False,
do_predict=False,
do_train=False,
eval_accumulation_steps=None,
eval_delay=0,
eval_do_concat_batches=True,
eval_steps=None,
evaluation_strategy=IntervalStrategy.NO,
fp16=False,
fp16_backend=auto,
fp16_full_eval=False,
fp16_opt_level=O1,
fsdp=[],
fsdp_config={'min_num_params': 0, 'xla': False, 'xla_fsdp_

### 训练过程中的指标评估（Evaluate)

**[Hugging Face Evaluate 库](https://huggingface.co/docs/evaluate/index)** 支持使用一行代码，获得数十种不同领域（自然语言处理、计算机视觉、强化学习等）的评估方法。 当前支持 **完整评估指标：https://huggingface.co/evaluate-metric**

训练器（Trainer）在训练过程中不会自动评估模型性能。因此，我们需要向训练器传递一个函数来计算和报告指标。

Evaluate库提供了一个简单的准确率函数，您可以使用`evaluate.load`函数加载

In [13]:
import numpy as np
import evaluate

metric = evaluate.load("accuracy")

2024-07-19 00:21:08.153571: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-07-19 00:21:08.181287: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-07-19 00:21:08.767203: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [14]:
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

In [15]:
from transformers import TrainingArguments, Trainer

training_args = TrainingArguments(output_dir=model_dir,
                                  evaluation_strategy="epoch",
                                  per_device_train_batch_size=32,
                                  num_train_epochs=3,
                                  logging_steps=30)

In [16]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=full_train_dataset,
    eval_dataset=full_eval_dataset,
    compute_metrics=compute_metrics,
)

In [17]:
full_test_dataset = tokenized_datasets["test"]

In [18]:
trainer.evaluate(full_test_dataset)

{'eval_loss': 0.7301458716392517,
 'eval_accuracy': 0.69574,
 'eval_runtime': 403.274,
 'eval_samples_per_second': 123.985,
 'eval_steps_per_second': 15.498}

増加到 69.574%